In [1]:
from transformers import LEDTokenizerFast

import pandas as pd
import numpy as np
from tqdm import tqdm

In [9]:
tokenizer = LEDTokenizerFast.from_pretrained("allenai/led-base-16384")
df = pd.read_csv('train-old.csv') # type: pd.DataFrame
df["box_center"] = None
df["box_length"] = None

In [10]:
df

id  discourse_id  discourse_start  discourse_end  \
0       423A1CA112E2  1.622628e+12              8.0          229.0   
1       423A1CA112E2  1.622628e+12            230.0          312.0   
2       423A1CA112E2  1.622628e+12            313.0          401.0   
3       423A1CA112E2  1.622628e+12            402.0          758.0   
4       423A1CA112E2  1.622628e+12            759.0          886.0   
...              ...           ...              ...            ...   
144288  4C471936CD75  1.618153e+12           2234.0         3203.0   
144289  4C471936CD75  1.618153e+12           3221.0         4509.0   
144290  4C471936CD75  1.618025e+12           4510.0         4570.0   
144291  4C471936CD75  1.618025e+12           4570.0         4922.0   
144292  4C471936CD75  1.618025e+12           4935.0         5825.0   

                                           discourse_text  \
0       Modern humans today are always on their phone....   
1       They are some really bad consequences when stu...   
2       Some certain areas in the United States ban ph...   
3       When people have phones, they know about certa...   
4       Driving is one of the way how to get around. P...   
...                                                   ...   
144288   if I'm not sure what college I want to attend...   
144289   seeking multiple opinions before making a har...   
144290  it is better to seek multiple opinions instead...   
144291  The impact of asking people to help you make a...   
144292  there are many other reasons one might want to...   

              discourse_type      discourse_type_num  \
0                       Lead                  Lead 1   
1                   Position              Position 1   
2                   Evidence              Evidence 1   
3                   Evidence              Evidence 2   
4                      Claim                 Claim 1   
...                      ...                     ...   
144288              Evidence              Evidence 2   
144289              Evidence              Evidence 3   
144290              Position              Position 1   
144291              Evidence              Evidence 4   
144292  Concluding Statement  Concluding Statement 1   

                                         predictionstring box_center  \
0       1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 1...       None   
1            45 46 47 48 49 50 51 52 53 54 55 56 57 58 59       None   
2         60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75       None   
3       76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 9...       None   
4       139 140 141 142 143 144 145 146 147 148 149 15...       None   
...                                                   ...        ...   
144288  386 387 388 389 390 391 392 393 394 395 396 39...       None   
144289  576 577 578 579 580 581 582 583 584 585 586 58...       None   
144290        828 829 830 831 832 833 834 835 836 837 838       None   
144291  839 840 841 842 843 844 845 846 847 848 849 85...       None   
144292  905 906 907 908 909 910 911 912 913 914 915 91...       None   

       box_length  
0            None  
1            None  
2            None  
3            None  
4            None  
...           ...  
144288       None  
144289       None  
144290       None  
144291       None  
144292       None  

[144293 rows x 10 columns]

In [11]:
for i in tqdm(range(len(df))):
  id = df.iloc[i]["id"]
  discourse_start = int(df.iloc[i]["discourse_start"])
  discourse_end = int(df.iloc[i]["discourse_end"])

  with open(f'train/{id}.txt', 'r') as f:
    text = f.read()

  char_array = np.zeros( (len(text)) )
  char_array[discourse_start : discourse_end] = 1

  encoding = tokenizer(text, padding='max_length', 
                      truncation=True, return_offsets_mapping=True)
  map_token_to_char = encoding['offset_mapping']

  len_sequence = len(map_token_to_char)

  token_array = np.zeros(len_sequence)
  for k in range(len(token_array)):
    token_array[k] = char_array[ map_token_to_char[k][0] - 1]

  pred = token_array.nonzero()[0]

  center = np.mean(pred) / len_sequence
  length = pred.shape[0] / len_sequence

  df.at[i, "box_center"] = center
  df.at[i, "box_length"] = length

  4%|▍         | 5536/144293 [01:14<41:31, 55.68it/s]  C:\Users\zumo0\anaconda3\envs\nlp\lib\site-packages\numpy\core\fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\zumo0\anaconda3\envs\nlp\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
100%|██████████| 144293/144293 [40:28<00:00, 59.41it/s] 


In [12]:
df

id  discourse_id  discourse_start  discourse_end  \
0       423A1CA112E2  1.622628e+12              8.0          229.0   
1       423A1CA112E2  1.622628e+12            230.0          312.0   
2       423A1CA112E2  1.622628e+12            313.0          401.0   
3       423A1CA112E2  1.622628e+12            402.0          758.0   
4       423A1CA112E2  1.622628e+12            759.0          886.0   
...              ...           ...              ...            ...   
144288  4C471936CD75  1.618153e+12           2234.0         3203.0   
144289  4C471936CD75  1.618153e+12           3221.0         4509.0   
144290  4C471936CD75  1.618025e+12           4510.0         4570.0   
144291  4C471936CD75  1.618025e+12           4570.0         4922.0   
144292  4C471936CD75  1.618025e+12           4935.0         5825.0   

                                           discourse_text  \
0       Modern humans today are always on their phone....   
1       They are some really bad consequences when stu...   
2       Some certain areas in the United States ban ph...   
3       When people have phones, they know about certa...   
4       Driving is one of the way how to get around. P...   
...                                                   ...   
144288   if I'm not sure what college I want to attend...   
144289   seeking multiple opinions before making a har...   
144290  it is better to seek multiple opinions instead...   
144291  The impact of asking people to help you make a...   
144292  there are many other reasons one might want to...   

              discourse_type      discourse_type_num  \
0                       Lead                  Lead 1   
1                   Position              Position 1   
2                   Evidence              Evidence 1   
3                   Evidence              Evidence 2   
4                      Claim                 Claim 1   
...                      ...                     ...   
144288              Evidence              Evidence 2   
144289              Evidence              Evidence 3   
144290              Position              Position 1   
144291              Evidence              Evidence 4   
144292  Concluding Statement  Concluding Statement 1   

                                         predictionstring box_center  \
0       1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 1...       None   
1            45 46 47 48 49 50 51 52 53 54 55 56 57 58 59       None   
2         60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75       None   
3       76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 9...       None   
4       139 140 141 142 143 144 145 146 147 148 149 15...       None   
...                                                   ...        ...   
144288  386 387 388 389 390 391 392 393 394 395 396 39...       None   
144289  576 577 578 579 580 581 582 583 584 585 586 58...       None   
144290        828 829 830 831 832 833 834 835 836 837 838       None   
144291  839 840 841 842 843 844 845 846 847 848 849 85...       None   
144292  905 906 907 908 909 910 911 912 913 914 915 91...       None   

       box_length    center    length  
0            None  0.001740  0.002930  
1            None  0.003723  0.000916  
2            None  0.004791  0.001099  
3            None  0.007568  0.004333  
4            None  0.010620  0.001648  
...           ...       ...       ...  
144288       None  0.032562  0.012817  
144289       None  0.048187  0.017700  
144290       None  0.057434  0.000793  
144291       None  0.060181  0.004700  
144292       None  0.069763  0.013855  

[144293 rows x 12 columns]

In [13]:
df.to_csv("train.csv")

In [21]:
dfna = df.fillna(0)

In [35]:
dfna[dfna['box_center'].isna()]

Empty DataFrame
Columns: [id, discourse_id, discourse_start, discourse_end, discourse_text, discourse_type, discourse_type_num, predictionstring, box_center, box_length]
Index: []

In [34]:
dfna.to_csv("train.csv")